In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM, AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

In [11]:
torch.manual_seed(42)

In [20]:
#model_path = '/scratch1/jhoff/checkpoints/generator_pythia-1B/runs/step_8000_merged'
#model_path = "EleutherAI/pythia-1B-Deduped"
#model_path = '/scratch1/jhoff/checkpoints/finetuned-pythia-6.9B-deduped/checkpoint_5000_merged'
#model_path = '/scratch1/jhoff/checkpoints/finetuned-pythia-6.9B-deduped/checkpoint_25000_merged'



questions = [
    "What is a^2 + b^2 = c^2?",
    'How does the rule of pythagoras work?',
    "Who is the president of the U.S.?",
    # "Why is the sky blue?", 
    # "Please explain climate change proof like I am 5", 
    # "What exactly is Obamacare and what did it change?", 
    'The 3:1 microphone rule', 
    
    "why are humans considered three dimensional beings when we also move through the fourth dimension of time?\nfor example, in interstellar and other sci-fi or theoretical scenarious we hear about 'four-dimensional beings'. But are humans not already 'four-dimensional beings' if we move through the fourth dimension that is time? ", 
    'Would there be any legal ramifications if a major company were to use an idea from a website like Reddit?\nFor example, Universal Studio uses an idea from r/writingprompt to create a film. If not, should we not create a system to protect out ideas from being stolen? ', 
    "What's the difference between blu-ray quality and 1080p? \nAlso, what is a common limiting factor of quality when using a tv or monitor? ", 
    'Why does blood taste like metal? ', 
    'Why do all of the new hires at my job suck? ', 
    'Best yo mama jokes please? ', 
    "Why can people sue Ashley Madison even though they weren't the ones that released the information? ", 
    'Toyota can\'t make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? ', 
]      

def test_model(model_path):

    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    model = model.cuda().eval()

    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        # "eos_token_id": 100_000,
        "pad_token_id": tokenizer.pad_token_id,
        "min_length": 32,
        "max_length": 128,
    }

    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')

    answers = {}

    for question in questions:
        result = pipeline(f"""Question: {question}\nAnswer:""", **generation_kwargs)
        result = result[0]['generated_text']
        print('-' * 30)
        print(f'{result}')
        print('-')
        print(f"Output length: {len(result)}")
        print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

        answers[question] = result

    return answers

In [21]:
# from reddit_dataset import load_reddit_dataset
#
# reddit_dataset = load_reddit_dataset("train")
# for question in questions:
#     is_in_dataset = reddit_dataset.filter(lambda x: x['question_title'] == question).num_rows > 0
#     print(f"Question: {question}\n  {is_in_dataset}")

In [22]:
answers_baseline = test_model('/scratch1/jhoff/llama-7b')
answers_trained_1000 = test_model('/scratch1/jhoff/checkpoints/llama-7b_new-dataset/checkpoint-1000_merged')
answers_trained_2000 = test_model('/scratch1/jhoff/checkpoints/llama-7b_new-dataset/checkpoint-2000_merged')
answers_trained_5000 = test_model('/scratch1/jhoff/checkpoints/llama-7b_new-dataset/checkpoint-5000_merged')

results = {
    'baseline': answers_baseline,
    'trained_1000': answers_trained_1000,
    'trained_2000': answers_trained_2000,
    'trained_5000': answers_trained_5000,
}

Loading checkpoint shards: 100%|██████████| 2/2 [01:10<00:00, 35.26s/it]
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


------------------------------
Question: What is a^2 + b^2 = c^2?
Answer: The equation is the area of the square, to be more precise, it is the the area of a regular quadrilateral, or nonagon, whose side length (b) is equal to the hypotenuse (c) squared.
The formula for the area of a square is a^2, where a is the length of the square side.
Clearly the length of such a side must be equal to the hypotenuse, and we want that hypotenuse to equal the sides squared. This leaves two
-
Output length: 449
Output tokens: 129


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


------------------------------
Question: How does the rule of pythagoras work?
Answer: Pythagoras rule works from the for-all statement.
Question: Which part of this is true?
Answer: a; b; c; d. Both Pythagoras Eg was 7 and Pythagoes Hg was 45.
Question: What is F=Phs.
Answer: To find the altitude the distance from the hypotenuse, the two legs.
Question. To find the height you must do the opposite?
Answer: the length of the missing side.

-
Output length: 411
Output tokens: 129


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

------------------------------
Question: What is a^2 + b^2 = c^2?
Answer: A and C tangent circles (equally likely to exist)
So, based on the fake photo:
A is the Sun,
C is Jupiter,
b is our orbital axis.
Tangent, spelled D, is a method used to find the Babylonian number π (pi). Instead of finding the 8th root of a number (where only two significant figures can be recorded), it used two different numbers continually squared to solve a recursive square root problem.
In the image
-
Output length: 450
Output tokens: 129
------------------------------
Question: How does the rule of pythagoras work?
Answer: The rule of pythagoras is the law of superposition.
The law of superposition says that the rocks on the surface layer of any rock unit (layer) were deposited or formed after those in the layer below it.
For the following diagrams, the light lines show the layers and the dark solid line is the line of pythagoras.
College town rocks
These are diagram for the same data. This time the light l

Loading checkpoint shards: 100%|██████████| 3/3 [02:42<00:00, 54.32s/it]


------------------------------
Question: What is a^2 + b^2 = c^2?
Answer: Is it (a + b)^2 = c^2? What happens if you cross out each term and see if you can relate the terms? What are the coefficients? What does the leading term mean?
If the terms are related like you have, then the answer is b.
The number 1 would never be shown as (1.0).
The answer to the following multiple choice question correctly identifies the units of measurement for areas.
triangular =?
A shape for which the side lengths are the lengths of
-
Output length: 486
Output tokens: 129
------------------------------
Question: How does the rule of pythagoras work?
Answer: The rule of pythagoras works by applying a certain ratio of the sides of a right triangle.
Question: Can you use the formula of pythagoras at any right triangle?
Answer: No, the formula works only with triangles that follow the right triangle index.
Question: Why the formula of pythagoras always works?
Answer: By defining a certain ratio that relates th

Loading checkpoint shards: 100%|██████████| 3/3 [02:58<00:00, 59.36s/it]


------------------------------
Question: What is a^2 + b^2 = c^2?
Answer: a^2 + b^2 = c^2 and (a^2 - b^2)^2 = c^2, but no value of a, b and c will set more than one formula (except for c = 0, none of the other ones give a^2 = b^2).
The parallelogram with area (a^2 + b^2) is seen in figure below. It rotates around the point M with a rotation axis perpendicular on the parallelogram and passing
-
Output length: 363
Output tokens: 129
------------------------------
Question: How does the rule of pythagoras work?
Answer: The squares of two of the three sides of a right triangle is equal to the square of hypotenuse.
The Proof :
Let ABC be a right Triangle then we can represent the statement of the problem in the form x^2 + y^2 + z^2 = 100 by x = 5, y=1 and Z=9.
Then the formula will be as follows:
(5^2) + (1^2) + (9^2) =100
(5^
-
Output length: 367
Output tokens: 129
------------------------------
Question: Who is the president of the U.S.?
Answer: Andrew Jackson
Your Final Thoughts
You ask 

In [23]:
results

{'baseline': {'What is a^2 + b^2 = c^2?': 'Question: What is a^2 + b^2 = c^2?\nAnswer: The equation is the area of the square, to be more precise, it is the the area of a regular quadrilateral, or nonagon, whose side length (b) is equal to the hypotenuse (c) squared.\nThe formula for the area of a square is a^2, where a is the length of the square side.\nClearly the length of such a side must be equal to the hypotenuse, and we want that hypotenuse to equal the sides squared. This leaves two',
  'How does the rule of pythagoras work?': 'Question: How does the rule of pythagoras work?\nAnswer: Pythagoras rule works from the for-all statement.\nQuestion: Which part of this is true?\nAnswer: a; b; c; d. Both Pythagoras Eg was 7 and Pythagoes Hg was 45.\nQuestion: What is F=Phs.\nAnswer: To find the altitude the distance from the hypotenuse, the two legs.\nQuestion. To find the height you must do the opposite?\nAnswer: the length of the missing side.\n',
  'Who is the president of the U.S.?

In [25]:
dfs = {}

for name, res in results.items():
    df = pd.DataFrame([{"question": q, "answer": a} for q,a in res.items()]).set_index('question')
    df.answer = df.answer.apply(lambda x: x.replace('\n', ' ').split('Answer: ')[1])
    df.rename(columns={'answer': name}, inplace=True)
    dfs[name] = df

df = pd.concat(dfs.values(), axis=1)

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None):
    display(df)

,baseline,trained_1000,trained_2000,trained_5000
question,,,,
What is a^2 + b^2 = c^2?,"The equation is the area of the square, to be more precise, it is the the area of a regular quadrilateral, or nonagon, whose side length (b) is equal to the hypotenuse (c) squared. The formula for the area of a square is a^2, where a is the length of the square side. Clearly the length of such a side must be equal to the hypotenuse, and we want that hypotenuse to equal the sides squared. This leaves two","A and C tangent circles (equally likely to exist) So, based on the fake photo: A is the Sun, C is Jupiter, b is our orbital axis. Tangent, spelled D, is a method used to find the Babylonian number π (pi). Instead of finding the 8th root of a number (where only two significant figures can be recorded), it used two different numbers continually squared to solve a recursive square root problem. In the image","Is it (a + b)^2 = c^2? What happens if you cross out each term and see if you can relate the terms? What are the coefficients? What does the leading term mean? If the terms are related like you have, then the answer is b. The number 1 would never be shown as (1.0). The answer to the following multiple choice question correctly identifies the units of measurement for areas. triangular =? A shape for which the side lengths are the lengths of","a^2 + b^2 = c^2 and (a^2 - b^2)^2 = c^2, but no value of a, b and c will set more than one formula (except for c = 0, none of the other ones give a^2 = b^2). The parallelogram with area (a^2 + b^2) is seen in figure below. It rotates around the point M with a rotation axis perpendicular on the parallelogram and passing"
How does the rule of pythagoras work?,Pythagoras rule works from the for-all statement. Question: Which part of this is true?,"The rule of pythagoras is the law of superposition. The law of superposition says that the rocks on the surface layer of any rock unit (layer) were deposited or formed after those in the layer below it. For the following diagrams, the light lines show the layers and the dark solid line is the line of pythagoras. College town rocks These are diagram for the same data. This time the light lines show the layers below the line of pythagoras and the",The rule of pythagoras works by applying a certain ratio of the sides of a right triangle. Question: Can you use the formula of pythagoras at any right triangle?,"The squares of two of the three sides of a right triangle is equal to the square of hypotenuse. The Proof : Let ABC be a right Triangle then we can represent the statement of the problem in the form x^2 + y^2 + z^2 = 100 by x = 5, y=1 and Z=9. Then the formula will be as follows: (5^2) + (1^2) + (9^2) =100 (5^"
Who is the president of the U.S.?,Barack Obama.,"Barack Obama Question: What is the capital of the United States? Why? First, you need to ask what you are talking about in the question: The capital of a state? The capital of a country? The capital of a city? Why do you ask that in this case? The answer is the capital of the country – Washington DC is the capital and not the capital of the state New York. This is a question specifically reserved to emigrants and to the U.S. but it is not asked in Europe.",Barack Obama Question for Winners: How often do you participate in interscholastic athletics?,"Andrew Jackson Your Final Thoughts You ask us to stop The Brush Fires, but nothing is being done. There has to be something we can do. The Brush Fires, 2014 The La Alianza Suicide Guide in Califor... The changing demeanor of the ICE Forever..."
The 3:1 microphone rule,"There's no specific ""rule"" but it is highly recommended to be in 3:1 microphone arrangement for quality... Q: What is the difference between microphone preamplifier and microphone amplifier? A: According to the International Standards Organization (ISO), a preamplifier is an amplifier on... Q: If your voice needs a soft approach in speech, what kind of ceiling microphone would you suggest? A: